# 5. Transacciones

Uno de los objetivos más importantes de los SGBD es garantizar la integridad de los datos almacenados en las BD que gestionan.

La integridad tiene que ver con la consistencia y la calidad de los datos. Hay diversas causas que pueden comprometer esa integridad: 
* el acceso simultáneo de usuarios diferentes a una misma BD, 
* una situación de avería, 
* el hecho de que se haya decidido tener datos replicados para mejorar el rendimiento en el acceso a la BD o 
* que una operación pueda comprometer una regla de inte- gridad definida sobre la BD.

En este apartado veremos las posibles anomalías que se derivan del acceso simultáneo de diversos usuarios a la misma BD y en el hecho de asegurar la disponibilidad de la BD ante fallos o desastres, como sería el caso de una avería en los dispositivos de almacenamiento externo, un apagón o un incendio.

Hay que tener presente que los datos de una organización casi siempre son uno de sus activos principales, una herramienta indispensable para el desarrollo normal de las actividades que lleva a cabo.
Por lo tanto, el SGBD tiene que afrontar todas estas posibles anomalías y, para hacerlo, se fundamenta en el concepto de transacción y en una serie de me- canismos para gestionar esas transacciones.

## 5.1. Problemática asociada a la gestión de transacciones

En los SGBD, el concepto de transacción representa la unidad de trabajo por lo que se refiere a control de concurrencia y recuperación. La gestión de transac- ciones que realiza el SGBD protege las aplicaciones de las anomalías impor- tantes que se pueden producir si no se lleva a cabo.

A continuación veremos, con ejemplos, los problemas que pueden surgir cuan- do se ejecutan de forma concurrente, y sin ningún control por parte del SGBD, diferentes transacciones.

Supongamos que una aplicación de una entidad bancaria ofrece a los usuarios una función que permite transferir cierta cantidad de dinero de una cuenta de origen a una cuenta de destino.

Esta función podría ejecutar los pasos que mostramos en la siguiente tabla (en pseudocódigo):


<img src="img2/13.png" width=550> 

Hay que considerar las anomalías que se producirán si no se toma ninguna precaución:
>1) Supongamos que un usuario empieza a ejecutar una de estas transferencias y, justo después del tercer paso, un apagón hace que el proceso no acabe. En este caso, se habrá restado la cantidad transferida al saldo de la cuenta de ori- gen, pero no se habrá sumado al de la cuenta de destino. Esta posibilidad re- presenta un peligro grave.

Desde el punto de vista de la aplicación, las operaciones que se ejecutan cuan- do se hace la transferencia se tienen que llevar a cabo completamente o no se tienen que efectuar en absoluto; es decir, la transferencia no puede quedar a medias.

>2) Supongamos que dos usuarios diferentes (A y B) intentan hacer dos transfe- rencias al mismo tiempo desde cuentas de origen diferentes y hacia la misma cuenta de destino. Analicemos qué puede pasar si, por cualquier motivo y sin ningún control por parte del SGBD, los pasos de las transacciones se ejecutan de forma concurrente en el siguiente orden:

<img src="img2/14.png" width=650> 
<img src="img2/15.png" width=650> 

El resultado final es que la cuenta de destino tiene como saldo la inicial más 40, en vez de más 60. Esto es incorrecto, ya que se ha perdido la cantidad que ha transferido el usuario A.

Hay que impedir de alguna manera que el acceso concurrente de diversos usua- rios produzca resultados anómalos.

Cada usuario, individualmente, tiene que tener la percepción de que solo él trabaja con la BD. En el ejemplo que hemos planteado, la ejecución de la trans- ferencia que efectúa el usuario B ha interferido en la ejecución de la transfe- rencia que lleva a cabo el usuario A. Si las dos transferencias se hubieran eje- cutado correctamente aisladas la una de la otra, el saldo total de la cuenta de destino habría sido el saldo inicial más 60.

>3) Imaginemos que un error de programación de la función de transferencia hace que el saldo de la cuenta de destino reciba como nuevo valor la cantidad que se ha transferido, en vez de sumarla al saldo anterior. Naturalmente, este comportamiento será incorrecto, ya que no se corresponde con el deseo de los usuarios, y dejará la BD en un estado inconsistente: los saldos que tendrían que tener las cuentas de acuerdo con los movimientos registrados (en el quinto paso) no coincidirían con los que se han almacenado realmente.
En conclusión, es misión de los diseñadores y los programadores que las transacciones verifiquen los requisitos de los usuarios.

>4) Planteémonos qué pasaría si, después de utilizar la aplicación durante unos cuantos días y en un momento de plena actividad, se produce un error fatal del dispositivo de almacenamiento externo en el que se guarda la BD, de manera que esta deja de estar disponible.
En definitiva, tiene que haber mecanismos para evitar la pérdida tanto de los datos más antiguos como de las actualizaciones más recientes.

## 5.2. Definición y propiedades de las transacciones

El acceso a los datos que hay en una BD se hace mediante la ejecución de operaciones en el SGBD correspondiente. Puesto que estamos interesados en SGBD relacionales, estas operaciones, a alto nivel, serán sentencias SQL. Ade- más, con vistas a resolver el tipo de problemas que hemos planteado en el apartado anterior, estas operaciones se agrupan en transacciones.

>Una __TRANSACCION__ es un conjunto de operaciones (de lectura y actualiza- ción) sobre la BD que se ejecutan como una __unidad indivisible de trabajo__. La transacción acaba su ejecución confirmando o cancelando los cambios que se han llevado a cabo sobre la BD

Un programa empieza a trabajar con una BD conectándose a ella de una ma- nera adecuada y estableciendo una sesión de trabajo que permite efectuar ope- raciones de lectura y actualización (inserciones, borrados, modificaciones) de la BD. Para hacer una operación tiene que haber una transacción activa (o en ejecución), que siempre es única. La transacción activa se puede iniciar me- diante una instrucción especial o automáticamente cuando se hace la primera operación en el SGBD.

Toda transacción debería cumplir cuatro propiedades, conocidas como propiedades ACID:
>1) __ATOMICIDAD__. El conjunto de operaciones que constituyen la transacción es la unidad atómica, indivisible, de ejecución. Esto quiere decir que, o bien se ejecutan todas las operaciones de la transacción (y, en este caso, la transacción confirma los resultados), o bien no se ejecuta ninguna en absoluto (y, en este caso, la transacción cancela los resultados). En definitiva, el SGBD tiene que garantizar el todo o nada para cada transacción:
>* a) Para confirmar los resultados producidos por la ejecución de una transac- ción, disponemos de la sentencia SQL __COMMIT__.
>
>* b) En el caso contrario, ya sea porque alguna cosa impide que se acabe de eje- cutar la transacción (por ejemplo, un corte de luz), ya sea porque la transac- ción acaba con una petición explícita de cancelación por parte del programa de aplicación, el SGBD tiene que deshacer todos los cambios que la transac- ción haya hecho sobre la BD hasta ese momento, como si dicha transacción nunca hubiera existido. En los dos casos se dice que la transacción ha abortado (en inglés, abort) la ejecución. Para cancelar de manera explícita los resultados producidos por la ejecución de una transacción, disponemos de la sentencia SQL __ROLLBACK__.

>2) __CONSISTENCIA__. La ejecución de una transacción tiene que preservar la con- sistencia de la BD. En otras palabras, si antes de ejecutarse una transacción la BD se encuentra en un estado consistente (es decir, en un estado en el que se verifican todas las reglas de integridad definidas sobre la BD), al acabar la eje- cución de la transacción la BD también tiene que quedar en un estado consis- tente, si bien, mientras la transacción esté activa, la BD podría caer momen- táneamente en un estado inconsistente.

>3) __AISLAMIENTO__. Una transacción no puede ver interferida su ejecución por ninguna otra transacción que se esté ejecutando de forma concurrente con esta. En definitiva, el SGBD tiene que garantizar el correcto aislamiento de las transacciones.

>4) __DEFINITIVIDAD__. Los resultados producidos por una transacción que confir- ma (es decir, que ejecuta la operación de COMMIT) tienen que ser definitivos en la BD; nunca se pueden perder, independientemente de que se produzcan fallos o desastres, hasta que otra transacción cambie esos resultados y los con- firme. Al contrario, los resultados producidos por una transacción que aborta su ejecución se han de descartar de la BD.

Es importante destacar que las propiedades que acabamos de presentar no son independientes entre ellas; por ejemplo, las propiedades de ato- micidad y de definitividad están estrechamente interrelacionadas. Ade- más, el hecho de garantizar las propiedades ACID de las transacciones no es solamente una misión del SGBD, sino también de las aplicaciones que lo utilizan y, por consiguiente, de su desarrollador.

## 5.3. Interferencias entre transacciones

En este apartado presentaremos los tipos de interferencias que se pueden pro- ducir si las transacciones que se ejecutan de manera concurrente no verifican la propiedad de aislamiento.
Antes de entrar en estas interferencias, es importante destacar que, si hay dos transacciones que se ejecutan de forma concurrente, una de estas sólo puede interferir en la ejecución de la otra si se dan las siguientes circunstancias:

>a) Las dos transacciones acceden a una misma porción de la BD.
>
>b) Como mínimo una de las dos transacciones, sobre esta porción común de la BD a la que acceden, efectúa operaciones de actualización.

En otras palabras, cuando las transacciones que se ejecutan de forma concu- rrente solo hacen lecturas, no se producirán nunca interferencias. De mane- ra similar, en el caso de que las transacciones hagan actualizaciones, si estas se realizan sobre porciones diferentes, no relacionadas en la BD, tampoco se pueden producir interferencias.


A continuación presentamos, mediante ejemplos, los tipos de interferencias que puede haber entre dos transacciones T1 y T2 que se procesan de forma concurrente si no están aisladas de una manera adecuada entre ellas:
>1) __ACTUALIZACION PERDIDA__  Esta interferencia se produce cuando se pierde un cambio efectuado por una transacción sobre un dato a causa de la presencia de otra transacción que también cambia el mismo dato. Esto podría suceder en una situación como la que se muestra a continuación:

<img src="img2/16.png" width=550>

T1 y T2 ejecutan un mismo tipo de transacción; en este caso, un reintegro de una misma cuenta bancaria. Las dos transacciones leen el mismo valor del saldo de la cuenta, lo actualizan de forma independiente (asumimos que hay saldo suficiente en la cuenta para hacer los reintegros) y restan a este saldo la cantidad que se ha sustraído.
Suponiendo que el SGBD ejecuta las operaciones que constituyen cada transac- ción sin ningún control y en el orden que se propone en el ejemplo, el cam- bio correspondiente a la sustracción de T1 se pierde. En consecuencia, el saldo disminuye sólo en 40, en vez de en 60.

En definitiva, T1 ha visto interferida su ejecución a causa de la presencia de T2. Si el orden de ejecución de las operaciones de cada transacción hubiera sido el siguiente:

<img src="img2/17.png" width=550>

se habría producido igualmente la interferencia. En este caso, se habría perdido el cambio efectuado por T2. En consecuencia, el saldo disminuiría solo en 20, en vez de en 60. En este caso, T2 habría visto interferida su ejecución a causa de la presencia de T1.

En definitiva, la interferencia ocurre porque se producen dos lecturas consecu- tivas de un mismo dato (el saldo de una misma cuenta) seguidas de dos cam- bios consecutivos del mismo dato (de nuevo, el saldo de una misma cuenta). Simplemente, si la secuencia de operaciones hubiera sido, por ejemplo, la que se muestra a continuación:

<img src="img2/18.png" width=550>

la interferencia no se habría producido. En este caso, T2 recuperaría el valor del saldo de cuenta dejado por T1 y, teniendo en cuenta este nuevo valor de saldo para la cuenta, efectuaría su propio reintegro. En consecuencia, el saldo de la cuenta disminuiría en 60.

>2) __LECTURA NO CONFIRMADA__. Esta interferencia se puede producir cuando una transacción recupera un dato pendiente de confirmación que ha sido modifi- cado por otra transacción que se ejecuta de forma concurrente con la transac- ción que recupera el dato. Eso podría suceder en diversas situaciones, como las que se muestran a continuación:

<img src="img2/19.png" width=550>
<img src="img2/20.png" width=550>

Primeramente, la transacción T2 lee el saldo de la cuenta y lo disminuye en la cantidad que se quiere reintegrar. A continuación, la transacción T1 efectúa una consulta de saldo de la misma cuenta sobre la que T2 hace el reintegro. El valor de saldo que obtiene T1 está pendiente de confirmar, es un dato pro- visional, ya que T2 todavía no ha confirmado sus resultados. Acto seguido, la transacción T1 finaliza su ejecución y confirma los resultados. Finalmente, T2 cancela su ejecución. Esta cancelación causa que los resultados producidos por T2 sean descartados de la BD, de manera que el saldo de la cuenta sea el que había antes de empezar la ejecución de T2. En consecuencia, T1 ha recuperado un valor que oficialmente nunca ha existido y ha visto interferida su ejecución por la transacción T2. Si las transacciones T1 y T2 hubieran estado aisladas correctamente, T1 nunca habría recuperado el valor provisional dejado por T2 y que finalmente ha sido descartado.

En el ejemplo previo, la interferencia de lectura no confirmada se produce a causa de la cancelación de la transacción que modifica los datos. Sin embargo, la interferencia se puede producir igualmente en el caso de que la transacción que modifica datos confirme los resultados.

Imaginemos ahora que tenemos dos transacciones, T1 y T2. La transacción T1 hace la consulta de un saldo de una cuenta corriente, mientras que T2 efectúa un par de reintegros de la misma cuenta corriente. Supongamos que el orden de ejecución de las operaciones es el que se muestra a continuación y que el SGBD no efectúa ningún control sobre el orden de ejecución de estas operaciones:

<img src="img2/21.png" width=550>

En este caso, y aunque la transacción T2 confirma los resultados, T1 ve inter- ferida su ejecución por T2 y recupera un dato provisional pendiente de confir- mación. Este dato corresponde a un saldo provisional para la cuenta corriente que corresponde al saldo que queda después del primer reintegro.

>3) __LECTURA REPETIBLE__. Esta interferencia se produce cuando una transac- ción, por los motivos que sea, necesita leer dos veces un mismo dato y en cada lectura recupera un valor diferente por el hecho de que hay otra transacción que se ejecuta simultáneamente y que efectúa una modificación del dato leí- do. Esto podría pasar en diversas situaciones, como la que se muestra a con- tinuación:

<img src="img2/22.png" width=550>

La transacción T2, que consulta dos veces el saldo de una misma cuenta co- rriente, recupera en cada lectura un valor diferente por el hecho de que la transacción T1, entre las dos lecturas, efectúa un reintegro e interfiere en la ejecución de la transacción T2. Si las transacciones se hubieran aislado co- rrectamente entre ellas, T2 habría recuperado el mismo valor para el saldo de cuenta corriente en las dos lecturas: o bien habría recuperado el valor que co- rrespondiera al saldo de la cuenta antes de que se efectuara el reintegro de T1, o bien, al saldo que quedara después de que se efectuara el reintegro de T1.

>4) __ANALISIS INCONSISTENTE (y el caso particular de fantasmas)__. Los tres tipos de interferencias anteriores se producen con respecto a un único dato de la BD, es decir, ocurren cuando dos transacciones intentan acceder a un mismo ítem de datos y, como mínimo, una de las dos transacciones modifica este ítem de datos. Pero también puede haber interferencias con respecto a la visión que dos transacciones tienen de un conjunto de datos que están interrelacionados.

Esto puede pasar, por ejemplo, cuando una transacción T1 lee unos datos mientras que otra transacción T2 actualiza una parte de ellos.

T1 puede obtener un estado de los datos incorrecto, como sucede con las si- guientes transacciones:

<img src="img2/23.png" width=550>

Los saldos que lee T1 no son correctos. No se corresponden ni con los de antes de la transferencia entre las dos cuentas ni con los de después, sino a un estado intermedio de T2 que no se tendría que haber visto nunca fuera del ámbito de T2. En consecuencia, T1 ha visto interferida su ejecución por T2.

Un caso particular bastante frecuente de esta interferencia son los fantasmas. Esta interferencia se puede producir cuando una transacción lee un conjunto de datos relacionado y hay otra transacción que dinámicamente cambia este conjunto de datos de interés añadiendo nuevos datos. Básicamente, la inter- ferencia ocurre cuando se produce la siguiente secuencia de acontecimientos:
* • UnatransacciónT1leeunaseriededatosquecumplenunacondiciónC determinada.
* • UnatransacciónT2insertanuevosdatosquecumplenlacondiciónC,o bien actualiza datos que no satisfacían la condición C y que ahora sí que la satisfacen.
* • LatransacciónT1vuelvealeerlosdatosquesatisfacenlacondiciónCo bien alguna información que depende de estos datos.
La consecuencia de esto es que T1 ve interferida su ejecución por T2 y encuen- tra un fantasma, es decir, unos datos que antes no cumplían la condición y que ahora sí que la cumplen. O también podría pasar que T1 no viera el fantasma directamente, sino el efecto que tiene en otros datos, tal como muestran los siguientes ejemplos:

<img src="img2/24.png" width=550>
<img src="img2/25.png" width=550>

En este primer ejemplo, la cuenta 4 es un fantasma desde el punto de vista de T1. Y además T1 ve el efecto que tiene en la suma de saldos que se produce y que, desde su punto de vista, da un resultado incoherente. Si T1 hubiera es- tado aislada correctamente de la transacción T2, una vez ejecutada la primera consulta, nunca tendría que haber encontrado los datos correspondientes a la cuenta 4.

Finalmente, el siguiente ejemplo muestra un fantasma que se produce a con- secuencia de una actualización de datos por parte de la transacción T2. Ima- ginemos que los propietarios de las cuentas 1 y 2 viven en Barcelona; los pro- pietarios de la cuenta 3 residen en Madrid, y los titulares de la cuenta 4, que vivían en Tarragona, notifican que ahora residirán en Barcelona.

<img src="img2/26.png" width=550>

En el ejemplo anterior, la cuenta 4 es nuevamente un fantasma desde el punto de vista de la transacción T1.

## 5.4. Nivel de concurrencia
Un SGBD puede resolver los problemas de interferencias entre transacciones que hemos visto anteriormente de dos maneras:

>a) Cancelar automáticamente (en inglés, abort) las transacciones problemáti-
cas y deshacer los cambios que han podido producir sobre la BD.
>
>b) Suspender la ejecución de una de las transacciones problemáticas tempo- ralmente y retomarla cuando haya desaparecido el peligro de interferencia. En algunos casos, esta situación también puede comportar la cancelación de transacciones.

Las dos soluciones implican un coste en términos de disminución del rendi- miento de la BD. Precisamente, en lo relativo a la gestión de transacciones, uno de los objetivos de los SGBD es minimizar estos efectos negativos.

>Se denomina nivel de concurrencia al grado de aprovechamiento de los recursos de proceso disponibles, según el solapamiento de la ejecu- ción de las transacciones que acceden de forma concurrente a la BD y se confirman.

El objetivo del SGBD es aumentar el trabajo efectivo (es decir, el trabajo real- mente útil para los usuarios) efectuado por unidad de tiempo. Sin duda, las transacciones que suspenden su ejecución no hacen trabajo efectivo, y todavía menos lo hacen las transacciones que finalmente cancelan su ejecución.

Uno de los grandes retos de la gestión de transacciones es alcanzar el nivel de concurrencia adecuado. Esto se consigue intentando que no se produzcan cancelaciones o suspensiones de ejecución de las transacciones cuando no es realmente necesario para impedir una interferencia. Desgraciadamente, este objetivo nunca se satisface del todo, ya que implicaría un esfuerzo excesivo y sería perjudicial para el rendimiento global por otros motivos. Los SGBD intentan obtener un compromiso óptimo entre el nivel de concurrencia que permiten y el coste que esto comporta en términos de tareas de control.


## 5.5. Visión externa de las transacciones
SQL estándar fuerza a que, una vez que se haya establecido una conexión con la BD, la primera sentencia SQL que queramos ejecutar mediante SQL interactivo __`implícitamente`__ inicie la ejecución de una transacción. Una vez iniciada la transacción, esta permanecerá activa hasta que __`explícitamente`__ y de una manera obligatoria indiquemos su finalización.

Por defecto, SQL estándar fuerza que esta transacción nunca vea interferida su ejecución y que tampoco pueda interferir en la ejecución de otras transac- ciones. En definitiva, por defecto, el SGBD deberá garantizar el correcto aisla- miento de todas las transacciones que accedan de forma concurrente a la BD.
Para informar sobre las características asociadas a una transacción desde SQL:1992, disponemos de la siguiente sentencia:

In [ ]:
SET TRANSACTION modo_acceso;

en la que __`modo_acceso`__ puede ser 
>* __READ ONLY__, en el caso de que la transacción solo consulte la BD, o 
>* __READ WRITE__, en el caso de que la transacción modifique la BD.

La sentencia previa solo se puede ejecutar en el caso de que no haya ninguna transacción en ejecución en la sesión de trabajo establecida con la BD; si hay alguna, SQL estándar especifica que el SGBD debería reportar una situación de error. Adicionalmente, las características especificadas serán aplicables al resto de transacciones que se ejecuten posteriormente durante la sesión de trabajo.

Para indicar la finalización de una transacción, SQL estándar nos ofrece la sentencia siguiente:

In [ ]:
{COMMIT | ROLLBACK} [WORK];

>__`COMMIT`__ confirma todos los cambios producidos contra la BD durante la ejecución de la transacción, 
>
>__`ROLLBACK`__ los deshace y deja la BD como estaba antes de que se iniciara la transacción. 
>
>__`La palabra reservada WORK`__ solo sirve para explicar qué hace la sentencia y es opcional.

__Ejemplos de uso de la sentencia SET TRANSACTION__
Supongamos que tenemos una BD de un banco que guarda datos de las cuentas de los clientes. En concreto, consideremos que tenemos la siguiente tabla (clave primaria sub- rayada):

In [ ]:
cuentas(num_cuenta, tipo_cuenta, saldo, comision)

Considerando que hemos establecido la conexión con la BD, podemos ejecutar las si- guientes sentencias durante nuestra sesión de trabajo con los efectos que se comentan:

<img src="img2/27.png" width=750>
<img src="img2/28.png" width=750>

El comienzo implícito de transacciones, en un entorno de aplicación real, pue- de crear confusiones sobre el alcance de cada transacción, si este alcance no se documenta correctamente. Por esto, desde SQL:1999 se propone utilizar la siguiente sentencia:

In [ ]:
START TRANSACTION (modo_acceso);

en la que __modo_acceso__ puede ser READ ONLY o READ WRITE. Si no se espe- cifica el modo de acceso, la sentencia simplemente inicia la ejecución de una nueva transacción, de acuerdo con las características que se hayan especificado previamente. Si antes no se ha especificado ninguna característica, SQL están- dar enuncia que la transacción se tiene que considerar de tipo READ WRITE.

__Ejemplos de uso de la sentencia START TRANSACTION__
En la BD del ejemplo anterior, y asumiendo que hemos establecido la conexión con la BD, podemos ejecutar las siguientes sentencias con los efectos que se comentan:

<img src="img2/29.png" width=750>

## 5.5.1. Relajación del nivel de aislamiento
Hasta ahora habíamos considerado que siempre era necesario garantizar una protección total ante cualquier tipo de interferencias. No obstante, esta pro- tección total exige una sobrecarga del SGBD en términos de gestión de infor- mación de control y una disminución del nivel de concurrencia.

En determinadas circunstancias, es conveniente relajar el nivel de aislamiento y posibilitar que se produzcan interferencias. Esto es correcto si se sabe que estas interferencias no ocurrirán realmente o si en el entorno de aplicación en el que nos encontramos no es importante que se produzcan.

Si nos centramos en SQL estándar, las instrucciones 
>__SET TRANSACTION__ y __START TRANSACTION__ permiten relajar el nivel de aislamiento. Tienen la si- guiente sintaxis:

In [ ]:
SET TRANSACTION {READ ONLY | READ WRITE},
ISOLATION LEVEL nivel_aislamiento;

START TRANSACTION [{READ ONLY | READ WRITE}],
ISOLATION LEVEL nivel_aislamiento;

>en la que nivel_aislamiento puede ser 
>* READ UNCOMMITTED, 
>* READ COMMITTED, 
>* REPEATABLE READ o 
>* SERIALIZABLE.

El nivel de aislamiento determina las interferencias que pueden desencade- nar otras transacciones en la transacción que empieza. De acuerdo con los ti- pos de interferencia que hemos descrito, la siguiente tabla indica las que se evitan con cada nivel de aislamiento:

<img src="img2/30.png" width=600>

En ella los niveles aparecen de menos a más estrictos y, por lo tanto, de menos a más eficientes:

>1) El nivel __READ UNCOMMITTED__  
protege los datos actualizados y evita que ninguna otra transacción los actualice hasta que no se acabe la transacción. No ofrece ninguna garantía con respecto a los datos que lea la transacción. Pueden ser datos actualizados por una transacción que todavía no ha confirmado y, además, otra transacción los puede actualizar inmediatamente.
>
>2) El nivel __READ COMMITTED__  
protege parcialmente las lecturas e impide que la transacción lea los datos actualizados por otra transacción que todavía no se hayan confirmado.
>
>3) El nivel __REPEATABLE READ__  
impide que otra transacción actualice un dato que haya leído la transacción hasta que esta no se acabe. De esta manera, la transacción puede volver a leer este dato sin riesgo de que lo hayan cambiado.
>
>4) El nivel __SERIALIZABLE__  
ofrece un aislamiento total y evita cualquier tipo de interferencias, incluyendo los fantasmas. Esto significa que no solamente protege los datos que haya visto la transacción, sino también cualquier infor- mación de control que se haya utilizado para hacer búsquedas.

La definición del SQL estándar establece que un SGBD concreto tiene la obli- gación de garantizar como mínimo el nivel de aislamiento que la transacción haya solicitado, aunque puede optar por ofrecer un aislamiento más restricti- vo. Por lo tanto, el único nivel que un SGBD tiene la obligación de implemen- tar es el más alto, el SERIALIZABLE.

### 5.5.2. Responsabilidades del SGBD y del desarrollador

Hemos visto qué es una transacción y qué propiedades tiene que cumplir. Exa- minemos la contribución del SGBD para conseguir garantizar estas propieda- des y los aspectos que dependen del desarrollador de las aplicaciones.

__1) Responsabilidades del SGBD__
>a) Conseguir que el __horario__ [la ejecución concurrente de un conjunto de transacciones (en las que se preserva el orden de las operaciones dentro de cada transacción) recibe el nombre de horario o historia.] que se produzca a medida que el SGBD recibe pe- ticiones de lectura o escritura, y de COMMIT o ROLLBACK de las transacciones que se ejecuten de forma concurrente sobre la BD, sea correcto (sin interferen- cias). Naturalmente, en el caso de que se haya relajado el nivel de aislamien- to para algunas transacciones, será necesario que el SGBD considere correctos más horarios.
>
>El SGBD consigue horarios libres de interferencias, sobre todo, de dos maneras (no necesariamente excluyentes entre ellas): 
>* cancelando automáticamente las transacciones problemáticas o 
>* suspendiendo la ejecución de la transacción hasta que la pueda retomar sin problemas. El conjunto de mecanismos que se responsabiliza de estas tareas se llama control de concurrencia.
>
>b) Comprobar que los cambios que ha hecho una transacción verifican todas las reglas de integridad que se han definido en la BD. Esto se puede hacer justo antes de aceptar el COMMIT de la transacción, rechazándolo si se viola alguna regla, o inmediatamente después de que se ejecute cada petición dentro de la transacción.
>
>c) Impedir que en la BD permanezcan cambios de transacciones que no se lle- gan a confirmar y que se pierdan los cambios que han llevado a cabo transac- ciones confirmadas en el caso de que se produzcan cancelaciones de transac- ciones, caídas del SGBD o de las aplicaciones, desastres (como incendios) o fallos de los dispositivos externos de almacenaje. En general, hablamos de re- cuperación para referirnos al conjunto de mecanismos que se encargan de estas tareas.

__2) Tareas del desarrollador de aplicaciones__
>a) Identificar con precisión las transacciones de una aplicación, es decir, el conjunto de operaciones que necesariamente se tiene que ejecutar de una ma- nera atómica sobre la BD de acuerdo con los requerimientos de los usuarios.
>
>En este sentido, las transacciones tendrían que durar el mínimo imprescindible. En concreto, puede ser muy peligroso que una aplicación tenga una transacción en ejecución mientras se espera la entrada de información por parte del usuario. A veces, los usuarios pueden tardar bastante rato en proporcionar ciertos datos o, simplemente, en apretar el botón de aceptación de un mensaje. Incluso es posible que cualquier circunstancia les haga dejar a medias lo que hacían y que la aplicación se quede bastante tiempo a la espera de que el usuario vuelva a ella. Hasta que el usuario no permite que la transacción se acabe, esta puede impedir la actualización o incluso la lectura de los datos a los que ya haya accedido. Esto significa más gasto de recursos y un freno importante en el nivel de concurrencia posible. Por lo tanto, y siempre que sea posible, __se suele recomendar que durante una transacción no se pare nunca la ejecución de la aplicación__ a la espera de que se produzca una actuación determinada por parte del usuario.

>b) Garantizar que las transacciones mantienen la consistencia de la BD de acuerdo con los requerimientos de los usuarios y teniendo en cuenta las restricciones de integridad y los disparadores definidos en la BD.
>
>c) Considerar aspectos de rendimiento. En particular, el desarrollador tiene que ser capaz de estudiar y mejorar el nivel de concurrencia de acuerdo con los conocimientos que tenga de los mecanismos de control de concurrencia del SGBD y las posibilidades de modificar su funcionamiento.

## 5.6. Transacciones en PostgreSQL

Por defecto, y si no se indica expresamente lo contrario, PostgreSQL trabaja con transacciones implícitas (este modo de trabajo también se conoce con el nombre de __autocommit activado__). Esto quiere decir que cualquier grupo de sen- tencias SQL que seleccionemos (por ejemplo, desde el PgAdmin) y enviemos a ejecutar será tratado como una transacción. Si el grupo de sentencias enviado no genera ningún error, los resultados pasarán a ser definitivos en la BD. De lo contrario, los resultados serán descartados por el SGBD.

Ya sabemos que trabajar con transacciones implícitas en un entorno de aplica- ción real puede crear confusiones sobre el alcance de cada transacción. Por es- to, PostgreSQL nos ofrece la sentencia de SQL estándar 
>__START TRANSACTION__, y también una sentencia propia, la sentencia __BEGIN__, para indicar de forma explícita el comienzo de una transacción. Cuando se indica explícitamente el comienzo de una transacción, PostgreSQL desactiva la modalidad autocommit y la transacción permanecerá activa hasta que confirmemos o cancelemos sus resultados de forma explícita. Para indicar la finalización de la transacción, disponemos de las sentencias de SQL estándar __COMMIT__ y __ROLLBACK__.

Adicionalmente, también tenemos disponible la sentencia 
>__SET TRANSACTION__ de SQL estándar para indicar las características de la transacción (si es READ ONLY o READ WRITE y el nivel de aislamiento) en el caso de que no se haya hecho anteriormente; por ejemplo, con las sentencias START TRANSACTION o BEGIN. Si el usuario no ha especificado ninguna característica para las transacciones que quiere ejecutar, por defecto, PostgreSQL considerará que son transacciones READ WRITE que trabajan con un nivel de aislamiento READ COMMITTED.

Aunque PostgreSQL permite especificar cualquiera de los niveles de aislamien- to propuestos por SQL estándar (READ UNCOMMITTED, READ COMMITTED, RE- PEATABLE READ y SERIALIZABLE), de hecho, internamente únicamente tra- baja con dos niveles de aislamiento.  
Estos niveles son los de READ COMMITTED y SERIALIZABLE:

>1) El nivel de aislamiento __READ COMMITTED__  
evita que la transacción se vea involucrada en interferencias de actualización perdida y de lectura no confir- mada. La transacción se podría ver implicada en interferencias de lectura no repetible y de análisis inconsistente (incluyendo fantasmas).
>
>2) Por su parte, el nivel de aislamiento __SERIALIZABLE__ 
evita cualquier tipo de interferencia.

El hecho de que PostgreSQL solo tenga que considerar internamente dos nive- les de aislamiento está relacionado con el mecanismo para el control de con- currencia que implementa. Este mecanismo se basa en lo que se conoce como modelo de control de concurrencia multiversión (en inglés, multiversion con- currency control, abreviado MVCC).


## 5.7. Importancia de las transacciones en OLTP frente a OLAP
Para finalizar, es importante destacar la importancia de las transacciones en sistemas OLTP en comparación con los sistemas OLAP. Para ello, utilizaremos como base la siguiente tabla con las principales diferencias entre estos dos sistemas.

<img src="img2/31.png" width=600>
 
 __OLTP__   On-line transaction processing  
 __OLAP__   On-line analytical processing  

__OLTP__  
Los sistemas OLTP están concebidos para resolver problemas concretos (tienen un propósito específico) y para ser utilizado en el día a día de las empresas, por lo que la carga de trabajo suele estar claramente predefinida. En estos sistemas, el acceso a los datos se realiza tanto para lectura como para escritura (inserción de datos nuevos y actualizaciones de datos existentes), con una complejidad de consultas por lo general simple (pocas tablas, combinaciones y agrupacio- nes) que manejan un conjunto de datos relativamente pequeño. 
>Por ejemplo, actualizar el nombre de un producto, actualizar el stock de un producto en concreto, obtener el listado de productos de una categoría en concreto, etc. Por último, el número de usuarios suele ser del orden de los miles o millones.
 
Por lo tanto, como ya hemos visto durante este capítulo y dadas las características de los entornos OLTP mencionadas, el uso de las transacciones resulta crítico para garantizar la consistencia de los datos y evitar así anomalías derivadas del acceso simultáneo de usuarios a la misma BD, o bien derivadas de fallos o desastres.

__OLAP__   
En contraposición, los sistemas OLAP están pensados para proporcionar análisis y dar soporte a la decisión, lo que significa que tienen una carga de trabajo impredecible, dependiendo de las necesidades del usuario final: 
>por ejemplo, a finales de año la carga de trabajo de estos sistemas podría incrementarse debido a la cantidad de información a procesar y la necesidad de, por ejemplo, realizar informes YoY. 
 
El acceso a la información en estos sistemas es de lectura exclusivamente, por lo que un usuario final de OLAP nunca realizará actualizaciones en los datos. Además, como podremos suponer, la complejidad de las consultas a la BD es mucho mayor respecto de los sistemas OLTP, ya que requiere generalmente de muchas agrupaciones y operaciones de combinación con muchas tablas, manejando grandes volúmenes de datos, como ya hemos comentado, históricos. Por último, el número de usuarios, al contrario que en OLTP, suele ser de unas decenas/cientos. Esto es así porque los usuarios finales son aquellos que requieren de información para realizar análisis y tomar decisiones, siendo estos generalmente analistas de negocio y directivos de empresa.

En consecuencia, al contrario que en OLTP, en los sistemas OLAP las transacciones no son necesarias para garantizar que los usuarios obtengan los datos consultados y, por ello, no tiene sentido hablar de transacciones en estos entornos, siempre desde un punto de vista del usuario final.

En cambio, es importante destacar que las transacciones sí son útiles (y nece- sarias en muchos casos) a la hora de desarrollar los procesos ETL que proporcionan información a los sistemas OLAP. Estos procesos ETL se encargan de leer información de los sistemas origen (que, entre otros, suelen ser los propios sistemas OLTP), información que, a través de tareas de transformación y de limpieza de datos, se carga en los sistemas OLAP. En estos casos, una transac- ción nos puede resultar de utilidad para, por ejemplo, garantizar la existencia de datos (recientes o no) tras un proceso de carga de datos. Por ejemplo, si una tabla se carga mediante un procedimiento de borrado y recarga (es decir, se eliminan todos los datos de la tabla y se vuelve a cargar de nuevo con el conte- nido más reciente), podríamos necesitar una transacción para garantizar que:

>1) Los datos antiguos han sido eliminados y los nuevos han sido cargados, en el caso de que la carga de estos últimos haya terminado correctamente sin errores (datos cargados y confirmados mediante una operación de COMMIT).
>
>2) En el caso de que haya sucedido un error en la carga de datos, los datos antiguos que se han borrado vuelvan a estar disponibles para evitar que los usuarios se queden sin información (cancelación de los cambios mediante una operación de ROLLBACK).